In [ ]:
#33335 Cutting_and_Creating_Putting_Videos
# Jupyter Notebook Setup

# Import necessary libraries
import os
import cv2
import json
import numpy as np
import subprocess
import pandas as pd
from datetime import datetime

# List of participant numbers
participant_numbers = ["059"]

# participant_numbers = [ "046", "062", "063", "064" , "065" ,"066" ,"067"]



# List of configurations for various conditions and hand types
configurations = [
    # {'condition': 'Ball', 'hand_type': 'BH'},
    {'condition': 'Ball', 'hand_type': 'RH'},
    # {'condition': 'Ball', 'hand_type': 'LH'}
    # {'condition': 'NB', 'hand_type': 'BH'},
    # {'condition': 'NB', 'hand_type': 'RH'},
    # {'condition': 'NB', 'hand_type': 'LH'}
    # Add more configurations as needed
]

# Function to extract video metadata
def extract_metadata(video_path):
    data = {'Total Frames': None, 'Timecode': None, 'Creation Time': None, 'Frame Rate': None, 'Audio Sample Rate': None, 'Fused Timecode': None, 'File Path': video_path}
    video_metadata_command = (f'ffprobe -v error -select_streams v:0 -show_entries stream=nb_frames,r_frame_rate,duration -show_entries stream_tags=timecode -show_entries format_tags=creation_time -of default=noprint_wrappers=1 "{video_path}"')
    audio_metadata_command = (f'ffprobe -v error -select_streams a:0 -show_entries stream=sample_rate -of default=noprint_wrappers=1 "{video_path}"')
    video_metadata_process = subprocess.run(video_metadata_command, shell=True, text=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    audio_metadata_process = subprocess.run(audio_metadata_command, shell=True, text=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if video_metadata_process.returncode == 0:
        for line in video_metadata_process.stdout.split('\n'):
            if 'nb_frames' in line:
                _, value = line.split('=')
                data['Total Frames'] = value.strip()
            elif 'TAG:timecode' in line:
                _, value = line.split('=')
                data['Timecode'] = value.strip()
            elif 'TAG:creation_time' in line:
                _, value = line.split('=')
                data['Creation Time'] = value.strip()
            elif 'r_frame_rate' in line:
                _, value = line.split('=')
                numerator, denominator = map(int, value.strip().split('/'))
                data['Frame Rate'] = str(numerator / denominator if denominator != 0 else numerator)
        if data['Creation Time'] and data['Timecode']:
            creation_datetime = datetime.strptime(data['Creation Time'], "%Y-%m-%dT%H:%M:%S.%fZ")
            time_parts = data['Timecode'].split(':')
            if len(time_parts) == 4:
                fused_timecode = f"{creation_datetime.strftime('%Y-%m-%d')}T{time_parts[0]}:{time_parts[1]}:{time_parts[2]}.{int(time_parts[3]):02d}Z"
                data['Fused Timecode'] = fused_timecode
    else:
        print("Error extracting video metadata:", video_metadata_process.stderr)
    if audio_metadata_process.returncode == 0:
        for line in audio_metadata_process.stdout.split('\n'):
            if 'sample_rate' in line:
                _, value = line.split('=')
                data['Audio Sample Rate'] = value.strip()
    else:
        print("Error extracting audio metadata:", audio_metadata_process.stderr)
    return data

# Function to find the closest frame
def find_closest_frame(target_index, frame_values):
    closest_frame_idx = None
    min_difference = float('inf')
    for idx, value in enumerate(frame_values):
        difference = abs(value - target_index)
        if difference < min_difference:
            min_difference = difference
            closest_frame_idx = idx
    print(f"Minimum difference for target index {target_index} is {min_difference}.")
    return closest_frame_idx, min_difference

# Iterate through each participant and configuration
for participant in participant_numbers:
    for config in configurations:
        condition = config['condition']
        hand_type = config['hand_type']

        # Paths for alignment and index data
        aligned_data_path = f"/Volumes/Beorn_4T/Experiment Data/SDC_{participant}/Time_Axis_Alignmen_Python.json"
        peak_indexes_path = f"/Volumes/Beorn_4T/Experiment Data/SDC_{participant}/Putt_Index_Locations.json"
        video_read_path = f"/Volumes/Beorn_4T/Experiment Data/SDC_{participant}/Videos/Videopanel/SDC_{participant}_{condition}_{hand_type}_VP.MP4"

        # Load aligned data
        with open(aligned_data_path, 'r') as json_file:
            Time_Axis_Alignment = json.load(json_file)

        # Load peak indexes
        with open(peak_indexes_path, 'r') as file:
            Index_Locations = json.load(file)

        # Extract index locations and time alignment data for the current condition and hand type
        ball_BH_indexes = Index_Locations[condition][hand_type]
        condition_data = Time_Axis_Alignment[condition]  # Access data for the specified condition
        hand_type_data = condition_data[hand_type]       # Access data for the specified hand type

        BH_time_block = hand_type_data['data']
        BH_time_block = [row[0] for row in BH_time_block if len(row) > 0]
        offset = hand_type_data['offset']

        # Open video and ensure it's accessible
        cap = cv2.VideoCapture(video_read_path)
        if not cap.isOpened():
            raise Exception(f"Error opening video file {video_read_path}.")

        # Define the output directory
        output_dir = f"/Volumes/Beorn_4T/Experiment Data/SDC_{participant}/Videos/Videopanel/cut_segments{condition}_{hand_type}/"
        os.makedirs(output_dir, exist_ok=True)

        # Amount to crop from the bottom
        crop_from_bottom = 600

        # Process each index
        for index in ball_BH_indexes:
            frame_number, min_difference = find_closest_frame(index, BH_time_block)
            if frame_number is not None and min_difference < 20:
                start_frame = max((frame_number) - 150, 0) - offset
                cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

                ret, first_frame = cap.read()
                if not ret:
                    print(f"Failed to read the frame at {start_frame}, skipping.")
                    continue

                # Crop the first frame from the bottom
                height, width = first_frame.shape[:2]
                cropped_height = max(height - crop_from_bottom, 0)
                cropped_first_frame = first_frame[:cropped_height, :]

                # Set up video writer
                output_filename = f"{output_dir}segment_{index}.mp4"
                fourcc = cv2.VideoWriter_fourcc(*'avc1')
                out = cv2.VideoWriter(output_filename, fourcc, 100, (int(cropped_first_frame.shape[1]), int(cropped_first_frame.shape[0])))

                out.write(cropped_first_frame)
                count = 1
                while count < 401:
                    ret, frame = cap.read()
                    if not ret:
                        print(f"Failed to read frame at count {count}.")
                        break
                    cropped_frame = frame[:cropped_height, :]
                    out.write(cropped_frame)
                    count += 1
                out.release()
            else:
                print(f"Skipping index {index}: No close frame found within tolerance or minimum difference too high.")

        cap.release()
        print(f"Video segments have been successfully extracted, cropped, and saved for participant {participant}, condition {condition}, and hand type {hand_type}.")

print("All configurations have been processed.")
